In [ ]:
import mne, os
from signal_processing.pre_process import *
from basic.arrange_data import read_files, create_results_folders
import numpy as np
from cmath import nan
import matplotlib.pyplot as plt
import pandas as pd

mne.set_log_level('error')

**Set the folder of EEG files**

Run the cell to be prompted with an input box for choosing the folder containing the EEG files of interest.

*NB! The path has to be in 'Data\Raw\' folder relative to this notebook.*

In [ ]:
exp_folder = r"Auditory Oddball/Baseline" # in case you want to just type the dir here
#exp_folder = input('Experiment folder (e.g., Eyes Open\Baseline):')

raw_folder = r"Data/Raw/OKTOS/"
clean_folder = r"Data/Clean/OKTOS"
dir_inprogress = os.path.join(raw_folder,exp_folder)
export_dir = os.path.join(clean_folder,exp_folder)

results_foldername = r"Results/OKTOS"
exp_condition = r"AO_T01"

file_dirs, subject_names = read_files(dir_inprogress,".bdf")

In [ ]:
tminmax = [-0.2, 0.8]
baseline_correction = (None,0)
event_dict = {'target after 1 standard': 17, 'target after 3 standards': 19,
                'target after 5 standards': 21, 'target after 7 standards': 23,
                'target after 9 standards': 25, 'target after 11 standards': 27}
target_ids = [17, 19, 21, 23, 25, 27]
button_id = 32

for i in range(len(file_dirs)):  
        # Read in the raw EEG data
        raw = mne.io.read_raw_bdf(file_dirs[i],infer_types=True,
                                eog=["EXG1","EXG2","EXG3","EXG4","EXG5","EXG6","EXG7","EXG8"],
                                stim_channel="Status")

        # Set the right montage (Biosemi32) and set reference as average across all channels
        raw = raw.set_montage(mne.channels.make_standard_montage("biosemi32")).load_data()\
                .set_eeg_reference(ref_channels='average')
        
        # Filtering and EOG-removal
        filt = raw.copy().load_data().filter(1,15,verbose=False)
        eog_projs, _ = mne.preprocessing.compute_proj_eog(filt,verbose=False,n_grad=0,n_mag=0,n_eeg=1,reject=None,no_proj=True,ch_name=["EXG1","EXG2","EXG3","EXG4","EXG5","EXG6","EXG7","EXG8"])
        filt.add_proj(eog_projs,remove_existing=True)
        filt.apply_proj()
        filt.drop_channels(["EXG1","EXG2","EXG3","EXG4","EXG5","EXG6","EXG7","EXG8"]) # later remove this from filter funct

        # Find events from the filtered EEG data and name them
        events = mne.find_events(filt,stim_channel='Status',consecutive=False,output='onset')
        buttontarget_events = []

        # Create an array of target tone events which have been responded with a button press (acknowledged deviant tones only)
        for m in range(len(events)):
                for n in target_ids:
                        if events[m][2] == n and events[m+1][2] == button_id:
                                buttontarget_events.append(events[m])
        buttontarget_events = np.asarray(buttontarget_events)

        # Plot all the events
        %matplotlib inline
        fig = mne.viz.plot_events(buttontarget_events, sfreq=filt.info['sfreq'],
                                first_samp=filt.first_samp,event_id=event_dict)
        fig.subplots_adjust(right=0.7)  # make room for legend

        # Create epochs time-locked to target tone events which are followed by button press
        picks = mne.pick_types(filt.info,eeg=True,stim=False)
        epochs = mne.Epochs(filt,buttontarget_events,event_id=event_dict,tmin=tminmax[0],tmax=tminmax[1],
                            baseline=baseline_correction,picks=picks,preload=True)
        
        # Plot the epochs' GFP plot before artefact rejection
        epochs.plot_image(title="GFP without AR ({})".format(subject_names[i]))

        # Use AutoReject to repair and remove epochs which are artefactual
        reject_criteria = get_rejection_threshold(epochs)
        print('Dropping epochs with rejection threshold:',reject_criteria)
        epochs.drop_bad(reject=reject_criteria)

        ar = AutoReject(thresh_method='random_search',random_state=1)
        ar.fit(epochs)
        epochs_ar, reject_log = ar.transform(epochs, return_log=True)

        reject_log.plot('horizontal')

        # Plot the epochs' GFP after artefact rejection
        epochs_ar.average().plot()
        epochs_ar.plot_image(title="GFP with AR ({})".format(subject_names[i]))

        # Display the final epochs object meta-data
        display(epochs_ar)

        # Save the cleaned EEG file as .fif file
        try:
                os.makedirs(export_dir)
        except FileExistsError:
                pass
        try:
                mne.Epochs.save(epochs_ar,fname='{}/{}_clean-epo.fif'.format(export_dir,subject_names[i]),overwrite=True)
        except FileExistsError:
                pass